In [20]:
import tensorflow as tf
import keras 
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import sys
import tensorflow.keras.backend as K
import random
import os                      
import numpy as np              
import pandas as pd             
import torch             
from PIL import Image, ImageOps, ImageFilter
import matplotlib.pyplot as plt 
import torch.nn as nn           
from torch.utils.data import DataLoader  
from PIL import Image          
import torch.nn.functional as F 
import torchvision.transforms as transforms   
from torchvision.utils import make_grid    
from torchvision.datasets import ImageFolder  
from torch.utils.data import random_split
import time
import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.regularizers import l2
# from keras import backend as K
from sklearn.utils import shuffle
from tensorflow.keras.layers import Lambda
import numpy.random as rng

%matplotlib inline

In [21]:
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions, preprocess_input
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout

In [22]:
from tensorflow.keras.models import load_model
base_model = load_model('/kaggle/input/modelh5/EFNetB1-ft-best.h5')
base_model.trainable = True

In [23]:
base_model.input_shape

(None, 240, 240, 3)

In [24]:
base_model = Model(inputs=base_model.input, outputs=base_model.get_layer("avg_pool").output)

In [25]:
base_model.output_shape

(None, 1280)

In [26]:
embeddings = base_model

In [27]:
main_dir = "/kaggle/input/rfcdata/reflection-connection-train-data/train"

In [28]:
class MapFunction():
    def __init__(self, imageSize):
        self.imageSize = imageSize
        
        
    def decode_and_resize(self, imagePath):
        image = tf.io.read_file(imagePath)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.convert_image_dtype(image, dtype=tf.float32)
        image = tf.image.resize(image, self.imageSize)
        return image
    
    
    def __call__(self, anchor, positive, negative):
        anchor = self.decode_and_resize(anchor)
        positive = self.decode_and_resize(positive)
        negative = self.decode_and_resize(negative)
        return (anchor, positive, negative)

In [29]:
class TripletGenerator:
    def __init__(self, datasetPath):
        self.peopleNames = list()
        for folderName in os.listdir(datasetPath):
            absoluteFolderName = os.path.join(datasetPath, folderName)
            numImages = len(os.listdir(absoluteFolderName))
            if numImages > 1:
                self.peopleNames.append(absoluteFolderName)
        self.allPeople = self.generate_all_people_dict()
        
        
    def generate_all_people_dict(self):
        allPeople = dict()
        for personName in self.peopleNames:
            imageNames = os.listdir(personName)
            personPhotos = [
                os.path.join(personName, imageName) for imageName in imageNames
            ]
            allPeople[personName] = personPhotos
        return allPeople
    
    
    def get_next_element(self):
        while True:
            anchorName = random.choice(self.peopleNames)
            temporaryNames = self.peopleNames.copy()
            temporaryNames.remove(anchorName)
            negativeName = random.choice(temporaryNames)
            (anchorPhoto, positivePhoto) = np.random.choice(
                a=self.allPeople[anchorName],
                size=2,
                replace=False
            )
            negativePhoto = random.choice(self.allPeople[negativeName])
            yield (anchorPhoto, positivePhoto, negativePhoto)

In [30]:
imageSize = (240,240,3)

def get_siamese_network(imageSize, embeddingModel):
    anchorInput = keras.Input(name="anchor", shape=imageSize)
    positiveInput = keras.Input(name="positive", shape=imageSize)
    negativeInput = keras.Input(name="negative", shape=imageSize)
    anchorEmbedding = embeddingModel(anchorInput)
    positiveEmbedding = embeddingModel(positiveInput)
    negativeEmbedding = embeddingModel(negativeInput)
    
    siamese_network = keras.Model(
        inputs=[anchorInput, positiveInput, negativeInput],
        outputs=[anchorEmbedding, positiveEmbedding, negativeEmbedding]
    )
    
    return siamese_network

In [31]:
class SiameseModel(keras.Model):
    def __init__(self, siameseNetwork, margin, lossTracker):
        super().__init__()
        self.siameseNetwork = siameseNetwork
        self.margin = margin
        self.lossTracker = lossTracker
        
    def _compute_distance(self, inputs):
        (anchor, positive, negative) = inputs
        embeddings = self.siameseNetwork((anchor, positive, negative))
        anchorEmbedding = embeddings[0]
        positiveEmbedding = embeddings[1]
        negativeEmbedding = embeddings[2]
        apDistance = tf.reduce_sum(
            tf.square(anchorEmbedding - positiveEmbedding), axis=-1
        )
        anDistance = tf.reduce_sum(
            tf.square(anchorEmbedding - negativeEmbedding), axis=-1
        )
        return (apDistance, anDistance)
    
    
    def _compute_loss(self, apDistance, anDistance):
        loss = apDistance - anDistance
        loss = tf.maximum(loss + self.margin, 0.0)
        return loss
    
    
    def call(self, inputs):
        (apDistance, anDistance) = self._compute_distance(inputs)
        return (apDistance, anDistance)
    
    
    def train_step(self, inputs):
        with tf.GradientTape() as tape:
            (apDistance, anDistance) = self._compute_distance(inputs)
            loss = self._compute_loss(apDistance, anDistance)
        gradients = tape.gradient(
            loss,
            self.siameseNetwork.trainable_variables)
        self.optimizer.apply_gradients(
            zip(gradients, self.siameseNetwork.trainable_variables)
        )
        self.lossTracker.update_state(loss)
        return {"loss": self.lossTracker.result()}
    
    
    def test_step(self, inputs):
        (apDistance, anDistance) = self._compute_distance(inputs)
        loss = self._compute_loss(apDistance, anDistance)
        self.lossTracker.update_state(loss)
        return {"loss": self.lossTracker.result()}
    
    
    @property
    def metrics(self):
        return [self.lossTracker]

In [32]:
trainTripletGenerator = TripletGenerator(datasetPath='/kaggle/input/rfcdata/reflection-connection-train-data/train')
valTripletGenerator = TripletGenerator(datasetPath='/kaggle/input/rfcdata/reflection-connection-train-data/test')

In [33]:
trainTfDataset = tf.data.Dataset.from_generator(
    generator=trainTripletGenerator.get_next_element,
    output_signature=(
        tf.TensorSpec(shape=(), dtype=tf.string),
        tf.TensorSpec(shape=(), dtype=tf.string),
        tf.TensorSpec(shape=(), dtype=tf.string),
    )
)

valTfDataset = tf.data.Dataset.from_generator(
    generator=valTripletGenerator.get_next_element,
    output_signature=(
        tf.TensorSpec(shape=(), dtype=tf.string),
        tf.TensorSpec(shape=(), dtype=tf.string),
        tf.TensorSpec(shape=(), dtype=tf.string),
    )
)

In [ ]:
mapFunction = MapFunction(imageSize=(240,240))
trainDs = (trainTfDataset
    .map(mapFunction)
    .shuffle(64)
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
)
valDs = (valTfDataset
    .map(mapFunction)
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
)

siameseNetwork =  get_siamese_network(
    imageSize = (240,240,3),
    embeddingModel=embeddings,
)

siameseModel = SiameseModel(
    siameseNetwork=siameseNetwork,
    margin=0.5,
    lossTracker=keras.metrics.Mean(name="loss"),
)

siameseModel.compile(
    optimizer=keras.optimizers.Adam(0.0001)
)

siameseModel.fit(
    trainDs,
    steps_per_epoch=50,
    validation_data=valDs,
    validation_steps=10,
    epochs=35,
)

if not os.path.exists('/kaggle/working/'):
    os.makedirs('/kaggle/working/')

modelPath = '/kaggle/working/'

keras.models.save_model(
    model=siameseModel.siameseNetwork,
    filepath=modelPath,
    include_optimizer=True,
)

Epoch 1/35


I0000 00:00:1710935996.141696     252 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


50/50 [==============================] - 120s 848ms/step - loss: 0.4969 - val_loss: 0.4733
Epoch 2/35
50/50 [==============================] - 36s 715ms/step - loss: 0.4874 - val_loss: 0.4590
Epoch 3/35
50/50 [==============================] - 36s 713ms/step - loss: 0.4890 - val_loss: 0.4988
Epoch 4/35
50/50 [==============================] - 36s 713ms/step - loss: 0.4832 - val_loss: 0.4896
Epoch 5/35
50/50 [==============================] - 36s 713ms/step - loss: 0.4642 - val_loss: 0.4174
Epoch 6/35
50/50 [==============================] - 36s 713ms/step - loss: 0.4585 - val_loss: 0.4532
Epoch 7/35
50/50 [==============================] - 36s 712ms/step - loss: 0.4350 - val_loss: 0.4536
Epoch 8/35
50/50 [==============================] - 36s 712ms/step - loss: 0.4342 - val_loss: 0.4359
Epoch 9/35
50/50 [==============================] - 36s 712ms/step - loss: 0.4376 - val_loss: 0.4108
Epoch 10/35
50/50 [==============================] - 36s 711ms/step - loss: 0.4397 - val_loss: 0.4545

In [ ]:
from tensorflow.keras.models import load_model

final_model = load_model("/kaggle/working/siamese_CL.h5", compile=False)
final_model.compile(optimizer='adam', loss=contrastive_loss)

In [ ]:
import json
import numpy as np

class ResultBuilder:
    def __init__(self):
        self.results = dict()
        
    def build(self, 
              query_image_labels: np.ndarray, 
              matched_labels: np.ndarray,   
              confidence_scores: np.ndarray):
    
        if len(query_image_labels.shape) != 1:
            raise ValueError(f'Expected query_image_labels to be 1-dimensional array, got {query_image_labels.shape} instead')
        
        if matched_labels.shape != (query_image_labels.shape[0],3):
            raise ValueError(f'Expected matched_labels to have shape {(query_image_labels.shape[0], 3)}, got {matched_labels.shape} instead')
        
        if confidence_scores.shape != (query_image_labels.shape[0],3):
            raise ValueError(f'Expected confidence_scores to have shape {(query_image_labels.shape[0], 3)}, got {confidence_scores.shape} instead')
            
        for i, x in enumerate(query_image_labels):
            labels = matched_labels[i]
            confidence = confidence_scores[i]
    
            result_x = [{'label': labels[j], 'confidence': confidence[j]} for j in range(0,3)]
    
            self.results.update({x: result_x})
        
        return self
    
    def to_json(self, path: str = '.') -> None:
        path = f'{path}/siamese_CL1.json'
        with open(path, 'w+') as f:
            json.dump(self.results, f)
    
    def __call__(self,
              query_image_labels: np.ndarray, 
              matched_labels: np.ndarray,   
              confidence_scores: np.ndarray,
              path: str = '.') -> None:
    
        self.build(query_image_labels, matched_labels, confidence_scores)
        self.to_json(path)

In [ ]:
def preprocess(image_path):
    image = Image.open(image_path).convert('RGB')
    width, height = image.size
    pad = 240
    image = image.resize((pad, pad))
    image = image.filter(ImageFilter.SHARPEN)
    image = np.array(image)
    image = np.expand_dims(image, axis=0)
    return image

In [ ]:
image_corpus = os.listdir("/kaggle/input/rfcdata/reflection-connection-test-data/image_corpus")
corpus = []

for i in image_corpus:
    x = preprocess("/kaggle/input/rfcdata/reflection-connection-test-data/image_corpus/"+i)
    corpus.append(x)
    
query = os.listdir("/kaggle/input/rfcdata/reflection-connection-test-data/query")
queries= []

for i in query:
    x = preprocess("/kaggle/input/rfcdata/reflection-connection-test-data/query/"+i)
    queries.append(x)

In [ ]:
index = 21
q = queries[index]
sims = []
for i in range(len(corpus)):
    sim = model.predict([q, corpus[i]], verbose= 0)
    sim=sim[0][0]
    sims.append((sim,i))
    
sims = sorted(sims)
sims

In [ ]:
sims[:3]

In [ ]:
def euclidean_to_similarity(euclidean_distance, scale_factor=1.0):
    return np.exp(-scale_factor * euclidean_distance)

In [ ]:
new_sims = []
for i in sims[:3]:
    similarity = euclidean_to_similarity(i[0])
    new_sims.append((similarity,i[1]))
new_sims

In [ ]:
matched_labels = []
confidence_scores = []

for i in range(len(queries)):
    q = queries[i]
    sims = []
    for j in range(len(corpus)):
        sim = model.predict([q, corpus[j]], verbose= 0)[0][0]
        sims.append((sim,j))
    sims = sorted(sims)
    new_sims = []
    for i in sims[:3]:
        similarity = euclidean_to_similarity(i[0])
        new_sims.append((similarity,i[1]))
        
    a = []
    b = []
    for entry in new_sims:
        a.append(image_corpus[entry[1]])
        b.append(float(entry[0]))
        
    matched_labels.append(a)
    confidence_scores.append(b)

In [ ]:
query_image_labels = np.array(query)
matched_labels = np.array(matched_labels)
confidence_scores = np.array(confidence_scores)

result_builder = ResultBuilder()
result_builder.build(query_image_labels, matched_labels, confidence_scores)
result_builder.to_json("/kaggle/working/")